In [4]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

def get_all_country_codes():
    url = "http://webservices.oorsprong.org/websamples.countryinfo/CountryInfoService.wso"
    headers = {
        "Content-Type": "text/xml; charset=utf-8",
        "SOAPAction": "http://www.oorsprong.org/websamples.countryinfo/ListOfCountryNamesByName"
    }
    body = """
    <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
      <soap:Body>
        <ListOfCountryNamesByName xmlns="http://www.oorsprong.org/websamples.countryinfo"/>
      </soap:Body>
    </soap:Envelope>"""
    response = requests.post(url, headers=headers, data=body)
    root = ET.fromstring(response.text)
    namespaces = {'soap': 'http://schemas.xmlsoap.org/soap/envelope/', 'web': 'http://www.oorsprong.org/websamples.countryinfo'}
    codes = [code.text for code in root.findall(".//web:sISOCode", namespaces)]
    return codes

def get_capital_city(country_code):
    url = "http://webservices.oorsprong.org/websamples.countryinfo/CountryInfoService.wso"
    headers = {
        "Content-Type": "text/xml; charset=utf-8",
        "SOAPAction": "http://www.oorsprong.org/websamples.countryinfo/CapitalCity"
    }
    body = f"""
    <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
      <soap:Body>
        <CapitalCity xmlns="http://www.oorsprong.org/websamples.countryinfo">
          <sCountryISOCode>{country_code}</sCountryISOCode>
        </CapitalCity>
      </soap:Body>
    </soap:Envelope>"""
    response = requests.post(url, headers=headers, data=body)
    root = ET.fromstring(response.text)
    namespaces = {'soap': 'http://schemas.xmlsoap.org/soap/envelope/', 'web': 'http://www.oorsprong.org/websamples.countryinfo'}
    capital = root.find(".//web:CapitalCityResult", namespaces)
    if capital is not None:
        return capital.text
    else:
        return None

# Fetch all country codes
country_codes = get_all_country_codes()

# Collect capital cities
capital_cities = {}
for code in country_codes:
    capital = get_capital_city(code)
    if capital:
        capital_cities[code] = capital

# Convert to DataFrame
df = pd.DataFrame(list(capital_cities.items()), columns=['CountryCode', 'CapitalCity'])
print(df)


    CountryCode CapitalCity
0            AX   Mariehamn
1            AF       Kabul
2            AL      Tirana
3            DZ     Algiers
4            AS   Pago Pago
..          ...         ...
241          EH    Laâyoune
242          WS        Apia
243          YE       Sanaa
244          ZM      Lusaka
245          ZW      Harare

[246 rows x 2 columns]
